In [1]:
from clifford.g4 import *
import numpy as np
from clifford import MultiVector
import numbers

/work/anaconda/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../anaconda/lib/python3.7/site-packages/clifford/__init__.py", line 83:
    @numba.njit(parallel=NUMBA_PARALLEL)
    def play_games():
    ^

  self.func_ir.loc))


In [2]:
# Using a normal 4d curved space
print(e1*e1)
print(e2*e2)
print(e3*e3)
print(e4*e4)

1.0
1.0
1.0
1.0


In [3]:
class DualMV:
    def __init__(self, internal_array=None):
        if internal_array is None:
            self.internal_array = [0*e1]
        else:
            self.internal_array = [+i for i in internal_array]
            
    def __add__(self, other):
        if issubclass(other.__class__, numbers.Number):
            new_internal_array = [+i for i in self.internal_array]
            new_internal_array[0] += other
            return DualMV(new_internal_array)
        if issubclass(other.__class__, MultiVector):
            new_internal_array = [+i for i in self.internal_array]
            new_internal_array[0] += other
            return DualMV(new_internal_array)
        elif issubclass(other.__class__, DualMV):
            new_internal_array = [0]*max([len(self.internal_array), len(other.internal_array)])
            for i in range(len(self.internal_array)):
                new_internal_array[i] += self.internal_array[i]
            for j in range(len(other.internal_array)):
                new_internal_array[j] += other.internal_array[j]
            return DualMV(new_internal_array)
    
    def __rmul__(self, other):
        if issubclass(other.__class__, numbers.Number):
            new_int_array = [other*i for i in self.internal_array]
            return DualMV(new_int_array)
    
    def __mul__(self, other):
        if issubclass(other.__class__, numbers.Number):
            new_int_array = [other*i for i in self.internal_array]
            return DualMV(new_int_array)
        if issubclass(other.__class__, MultiVector):
            new_int_array = [0]*len(self.internal_array)
            for i in range(len(self.internal_array)):
                new_int_array[i] = self.internal_array[i]*other
            return DualMV(new_int_array)
        elif issubclass(other.__class__, DualMV):
            output = DualMV()
            for i in range(len(other.internal_array)):
                output += DualMV([0*e1]*i + (self*other.internal_array[i]).internal_array)
            return output
        
    def __xor__(self, other):
        if issubclass(other.__class__, MultiVector):
            new_int_array = [0]*len(self.internal_array)
            for i in range(len(self.internal_array)):
                new_int_array[i] = self.internal_array[i]^other
            return DualMV(new_int_array)
        elif issubclass(other.__class__, DualMV):
            output = DualMV()
            for i in range(len(other.internal_array)):
                output += DualMV([0*e1]*i + (self^other.internal_array[i]).internal_array)
            return output
        
    def __pow__(self, other):
        new = DualMV([1])
        for i in range(other):
            new = new*self
        return new
    
    def __str__(self):
        total_string = ''
        for i in range(len(self.internal_array)):
            if i > 0:
                total_string +='d' + str(i) + '*[' + str(self.internal_array[i]) + ']'
            else:
                total_string += str(self.internal_array[i])
            if i < len(self.internal_array) -1:
                total_string+= ' + '
        return total_string
    
    def __repr__(self):
        return str(self)
    
    @property
    def minterm(self):
        for i in range(len(self.internal_array)):
            if self.internal_array[i] != 0*e1:
                return i
    
    def trim(self, order=1):
        """
        Removes anything above the order specified
        """
        return DualMV([i for i in self.internal_array[0:order+1]])
            
    def homo(self):
        minterm = self.minterm
        coef = (self.internal_array[minterm]|e4)[0]
        new_dnum = DualMV([i/coef for i in self.internal_array[minterm:]])
        return new_dnum
    
    def __invert__(self):
        return DualMV([~i for i in self.internal_array])
        
    

In [4]:
# Create the dual unit
d1 = DualMV([0*e1, 0*e1 + 1.0])
d1

0 + d1*[1.0]

In [5]:
# This is the pseudo scalar (for the duals)
I4 = e1234

In [6]:
# Some utility functions
def up(x):
    return 2*d1*x + e4

def down(x):
    return 0.5*(x.homo().internal_array[1])

def meet(X, Y):
    return ((X*I4)^(Y*I4))*I4

def random_euc_point():
    return random.randn()*e1 + random.randn()*e2 + random.randn()*e3

def random_point():
    return up(random_euc_point())

# Lets have a go with intersecting a plane and a line

In [7]:
# Create some points
A = up(1.0*e1)
B = up(1.0*e2)
C = up(1.0*e1 + 1.0*e2 + 1.0*e3)
print(A)
print(B)
print(C)

(1.0^e4) + d1*[(2.0^e1)]
(1.0^e4) + d1*[(2.0^e2)]
(1.0^e4) + d1*[(2.0^e1) + (2.0^e2) + (2.0^e3)]


In [8]:
# Wedge the points to give a line and a plane
L1 = A^C
P2 = B^C^up(0*e1)
print(L1)
print(P2)

0 + d1*[-(2.0^e24) - (2.0^e34)] + d2*[(4.0^e12) + (4.0^e13)]
0 + d1*[0] + d2*[-(4.0^e124) + (4.0^e234)] + d3*[0]


In [9]:
# Perform the meet as usual with the dual and wedge
meet(L1,P2)

0 + d1*[0] + d2*[0] + d3*[(8.0^e4)] + d4*[(16.0^e1) + (16.0^e2) + (16.0^e3)] + d5*[0]

In [10]:
# Calculate the minimum order term of d
meet(L1,P2).minterm

3

In [11]:
# Homogenise to get the point at the intersection
meet(L1,P2).homo()

(1.0^e4) + d1*[(2.0^e1) + (2.0^e2) + (2.0^e3)] + d2*[0]

In [12]:
# Call down to convert back to a normal euc point
down(meet(L1,P2))

(1.0^e1) + (1.0^e2) + (1.0^e3)

# Translators seem to work as well

In [13]:
def translation_rotor(t):
    return (d1*t)*e4 + 1

In [14]:
R = translation_rotor(10*e1+5*e3)
print(R)
print(R*~R)

1.0 + d1*[(10.0^e14) + (5.0^e34)]
1.0 + d1*[0] + d2*[125.0]


In [15]:
R*C*~R

(1.0^e4) + d1*[(22.0^e1) + (2.0^e2) + (12.0^e3)] + d2*[-(185.0^e4)] + d3*[-(350.0^e1) + (250.0^e2) - (50.0^e3)]

In [16]:
print(down(C))
print(down((R*C*~R).trim()))

(1.0^e1) + (1.0^e2) + (1.0^e3)
(11.0^e1) + (1.0^e2) + (6.0^e3)


# Rotation rotors are the same as in 3D

In [17]:
def rotation_rotor(m,n,theta):
    B = (m*n).normal()
    rotor = np.cos(theta / 2) - B * np.sin(theta / 2)
    return DualMV([rotor])

In [1]:
R = rotation_rotor(e1,e2,np.pi/4)
print(R)
print(R*~R)

NameError: name 'rotation_rotor' is not defined

In [19]:
R*(up(e1)*~R)

(1.0^e4) + d1*[(1.41421^e1) + (1.41421^e2)]

1.0

In [21]:
down(R*(up(e1)*~R)) # This is wrong..

(0.70711^e1) + (0.70711^e2)

In [22]:
(R*up(e1))*~R

(1.0^e4) + d1*[(1.41421^e1) + (1.41421^e2)]

In [23]:
R*e1*~R

(0.70711^e1) + (0.70711^e2)

In [24]:
R

0.92388 - (0.38268^e12)

In [25]:
down((R*C*~R))

(1.41421^e2) + (1.0^e3)

In [27]:
down((R*(C*I4)*~R)*I4)

(1.41421^e2) + (1.0^e3)